# Iris

## Read the data

In [1]:
import iris
import numpy

In [5]:
infile = "/g/data/ua6/drstree/CMIP5/GCM/CSIRO-BOM/ACCESS1-0/historical/mon/ocean/thetao/r1i1p1/thetao_Omon_ACCESS1-0_historical_r1i1p1_186501-186912.nc"

In [3]:
def lat_subset(cell):
    return -10 < cell < 10

def lon_subset(cell):
    return 30 < cell < 70

lat_constraint = iris.Constraint(latitude=lat_subset)
lon_constraint = iris.Constraint(longitude=lon_subset)

If I want to process the data on the native model grid, then you can't subset via lat and lon...

In [4]:
with iris.FUTURE.context(cell_datetime_objects=True):
    cube = iris.load_cube(infile, 'sea_water_potential_temperature' & lat_constraint & lon_constraint)

/g/data/r87/dbi599/miniconda2/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:794: UserWarning: Missing CF-netCDF measure variable u'areacello', referenced by netCDF variable u'thetao'
  warnings.warn(message % (variable_name, nc_var_name))
/g/data/r87/dbi599/miniconda2/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:794: UserWarning: Missing CF-netCDF measure variable u'volcello', referenced by netCDF variable u'thetao'
  warnings.warn(message % (variable_name, nc_var_name))
/g/data/r87/dbi599/miniconda2/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:1139: UserWarning: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warnings.warn(msg)


CoordinateMultiDimError: Cannot apply constraints to multidimensional coordinates

But by depth is fine...

In [5]:
level_subset = lambda cell: cell <= 50
level_constraint = iris.Constraint(depth=level_subset)

In [6]:
with iris.FUTURE.context(cell_datetime_objects=True):
    cube = iris.load_cube(infile, 'sea_water_potential_temperature' & level_constraint)

In [7]:
print cube

sea_water_potential_temperature          (time: 60; depth: 6; cell index along second dimension: 300; cell index along first dimension: 360)
     Dimension coordinates:
          time                                  x          -                                     -                                      -
          depth                                 -          x                                     -                                      -
          cell index along second dimension     -          -                                     x                                      -
          cell index along first dimension      -          -                                     -                                      x
     Auxiliary coordinates:
          latitude                            -          -                                     x                                      x
          longitude                           -          -                                     x                   

In [31]:
coords = cube.coords(dim_coords=True)

In [33]:
print type(coords[0])
print type(coords[-1])
print len(coords)

<class 'iris.coords.DimCoord'>
<class 'iris.coords.DimCoord'>
4


In [36]:
aux_coords = cube.aux_coords

In [38]:
print type(aux_coords[0])
print len(aux_coords)

<class 'iris.coords.AuxCoord'>
2


In [80]:
print cube.metadata
print cube.var_name

CubeMetadata(standard_name=u'sea_water_potential_temperature', long_name=u'Sea Water Potential Temperature', var_name=u'thetao', units=Unit('K'), attributes={'initialization_method': 1, 'product': 'output', 'experiment': 'historical', 'model_id': 'ACCESS1-0', 'parent_experiment_rip': 'r1i1p1', 'history': "2012-01-15T15:26:36Z altered by CMOR: Converted units from 'deg_C' to 'K'. 2012-01-15T15:26:36Z altered by CMOR: replaced missing value flag (-1e+20) with standard missing value (1e+20).", 'parent_experiment_id': 'piControl', 'cmor_version': '2.8.0', 'creation_date': '2012-01-15T15:26:38Z', 'frequency': 'mon', 'references': 'See http://wiki.csiro.au/confluence/display/ACCESS/ACCESS+Publications', 'parent_experiment': 'pre-industrial control', 'institution': 'CSIRO (Commonwealth Scientific and Industrial Research Organisation, Australia), and BOM (Bureau of Meteorology, Australia)', 'experiment_id': 'historical', 'version_number': 'v20120115', 'forcing': 'GHG, Oz, SA, Sl, Vl, BC, OC, (

In [39]:
lev_bounds = cube.coord('depth').bounds
lev_diffs = numpy.apply_along_axis(lambda x: x[1] - x[0], 1, lev_bounds)

## Calculate something

In [40]:
def simple_integration(vert_vector, level_steps):
    """Perform vertical integration for a single lat/lon/time point."""
    
    areas = vert_vector * level_steps
    
    return areas.sum()

In [41]:
integral = numpy.ma.apply_along_axis(simple_integration, 1, cube.data, lev_diffs)

## Rebuild the cube and write to file

[This page](http://esc24.github.io/iris/iris/iris/cube.html#iris.cube.Cube) tells you how. An important point is that if the standard name you want to use isn't in `iris.std_names.STD_NAMES` then you need to add it.

In [61]:
valid_names = iris.std_names.STD_NAMES.keys()
for name in valid_names:
    if 'heat' in name:
        print name

downward_heat_flux_in_sea_ice
upward_heat_flux_in_sea_water_due_to_convection
upward_geothermal_heat_flux_at_sea_floor
northward_heat_flux_due_to_eddy_advection
northward_ocean_heat_transport_due_to_diffusion
downward_heat_flux_at_ground_level_in_soil
upward_sensible_heat_flux_in_air
y_heat_flux_in_sea_water_due_to_advection
atmosphere_heat_diffusivity
surface_upward_sensible_heat_flux_where_sea
northward_ocean_heat_transport_due_to_gyre
tendency_of_mass_fraction_of_stratiform_cloud_liquid_water_in_air_due_to_condensation_and_evaporation_from_longwave_heating
surface_snow_melt_and_sublimation_heat_flux
surface_upward_sensible_heat_flux
ocean_vertical_heat_diffusivity
ocean_heat_x_transport_due_to_bolus_advection
upward_heat_flux_at_ground_level_in_snow
ocean_heat_y_transport
tendency_of_air_temperature_due_to_longwave_heating_assuming_clear_sky
surface_drag_coefficient_for_heat_in_air
upward_sea_ice_basal_heat_flux
integral_of_surface_downward_sensible_heat_flux_wrt_time
surface_downwa

In [63]:
iris.std_names.STD_NAMES['downward_heat_flux_in_sea_ice']

{'canonical_units': 'W m-2'}

In [64]:
iris.std_names.STD_NAMES['ocean_heat_content'] = {'canonical_units': 'J m-2'}

In [65]:
type(integral)

numpy.ma.core.MaskedArray

In [82]:
out_cube = iris.cube.Cube(integral,
                          standard_name='ocean_heat_content',
                          long_name='ocean heat content',
                          var_name='ohc',
                          units='J m-2',
                          attributes=cube.attributes,
                          dim_coords_and_dims=[(coords[0], 0), (coords[2], 1), (coords[3], 2)],
                          aux_coords_and_dims=[(aux_coords[0], [1, 2]), (aux_coords[1], [1, 2])],
                         )

In [83]:
print out_cube

ocean_heat_content                         (time: 60; cell index along second dimension: 300; cell index along first dimension: 360)
     Dimension coordinates:
          time                                  x                                      -                                      -
          cell index along second dimension     -                                      x                                      -
          cell index along first dimension      -                                      -                                      x
     Auxiliary coordinates:
          latitude                              -                                      x                                      x
          longitude                             -                                      x                                      x
     Attributes:
          Conventions: CF-1.4
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_ACCESS1-0

In [84]:
print out_cube.metadata

CubeMetadata(standard_name='ocean_heat_content', long_name='ocean heat content', var_name='ohc', units=Unit('J m-2'), attributes={'project_id': 'CMIP5', 'initialization_method': 1, 'product': 'output', 'tracking_id': '4b444b5a-95ce-47b3-8138-5a7811823450', 'model_id': 'ACCESS1-0', 'parent_experiment_id': 'piControl', 'cmor_version': '2.8.0', 'creation_date': '2012-01-15T15:26:38Z', 'frequency': 'mon', 'references': 'See http://wiki.csiro.au/confluence/display/ACCESS/ACCESS+Publications', 'institute_id': 'CSIRO-BOM', 'modeling_realm': 'ocean', 'parent_experiment': 'pre-industrial control', 'institution': 'CSIRO (Commonwealth Scientific and Industrial Research Organisation, Australia), and BOM (Bureau of Meteorology, Australia)', 'version_number': 'v20120115', 'forcing': 'GHG, Oz, SA, Sl, Vl, BC, OC, (GHG = CO2, N2O, CH4, CFC11, CFC12, CFC113, HCFC22, HFC125, HFC134a)', 'original_units': 'deg_C', 'title': 'ACCESS1-0 model output prepared for CMIP5 historical', 'physics_version': 1, 'Conv

In [86]:
iris.save(out_cube, "myfile.nc")

# xarray

## Read the data

In [3]:
import xray
import numpy

In [4]:
dset = xray.open_dataset(infile)

NameError: name 'infile' is not defined

In [91]:
print dset

<xarray.Dataset>
Dimensions:       (bnds: 2, i: 360, j: 300, lev: 50, time: 60, vertices: 4)
Coordinates:
  * time          (time) datetime64[ns] 1865-01-16T12:00:00 1865-02-15 ...
  * lev           (lev) float64 5.0 15.0 25.0 35.0 45.0 55.0 65.0 75.0 85.0 ...
  * j             (j) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
  * i             (i) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
    lat           (j, i) float32 -77.8766 -77.8766 -77.8766 -77.8766 ...
    lon           (j, i) float32 80.5 81.5 82.5 83.5 84.5 85.5 86.5 87.5 ...
  * bnds          (bnds) int64 0 1
  * vertices      (vertices) int64 0 1 2 3
Data variables:
    time_bnds     (time, bnds) float64 6.808e+05 6.808e+05 6.808e+05 ...
    lev_bnds      (lev, bnds) float64 0.0 10.0 10.0 20.0 20.0 30.0 30.0 40.0 ...
    lat_vertices  (j, i, vertices) float32 -78.0 -78.0 -77.7532 -77.7532 ...
    lon_vertices  (j, i, vertices) float32 80.0 81.0 81.0 80.0 81.0 82.0 ...
    thetao        (time, lev,

In [110]:
dset.coords.keys()

[u'time', u'lev', u'j', u'i', u'lat', u'lon', u'bnds', u'vertices']

In [113]:
dset.coords['time'].values

array(['1865-01-16T12:00:00.000000000', '1865-02-15T00:00:00.000000000',
       '1865-03-16T12:00:00.000000000', '1865-04-16T00:00:00.000000000',
       '1865-05-16T12:00:00.000000000', '1865-06-16T00:00:00.000000000',
       '1865-07-16T12:00:00.000000000', '1865-08-16T12:00:00.000000000',
       '1865-09-16T00:00:00.000000000', '1865-10-16T12:00:00.000000000',
       '1865-11-16T00:00:00.000000000', '1865-12-16T12:00:00.000000000',
       '1866-01-16T12:00:00.000000000', '1866-02-15T00:00:00.000000000',
       '1866-03-16T12:00:00.000000000', '1866-04-16T00:00:00.000000000',
       '1866-05-16T12:00:00.000000000', '1866-06-16T00:00:00.000000000',
       '1866-07-16T12:00:00.000000000', '1866-08-16T12:00:00.000000000',
       '1866-09-16T00:00:00.000000000', '1866-10-16T12:00:00.000000000',
       '1866-11-16T00:00:00.000000000', '1866-12-16T12:00:00.000000000',
       '1867-01-16T12:00:00.000000000', '1867-02-15T00:00:00.000000000',
       '1867-03-16T12:00:00.000000000', '1867-04-16

## Calculate something

In [99]:
vert_mean = numpy.ma.array(dset['thetao'].mean(dim='lev'))

In [2]:
print type(vert_mean)
print vert_mean

NameError: name 'vert_mean' is not defined

In [1]:
vert_mean.mask

NameError: name 'vert_mean' is not defined

In [ ]:
d = {}
d['time'] = ('time', time_coord.points)
    d['latitude'] = ('latitude', lat_coord.points)
    d['longitude'] = ('longitude', lon_coord.points)
    d['ohc'] = (['time', 'latitude', 'longitude'], ohc_per_m2)

    dset_out = xray.Dataset(d)
    notes_text = 'OHC integrated over %s' %(domain_text(lev_coord.points, 
                                                        inargs.min_depth,
                                                        inargs.max_depth))
    dset_out['ohc'].attrs =   {'standard_name': 'ocean_heat_content',
                               'long_name': 'ocean_heat_content',
                               'units': '10^%d J m-2' %(inargs.scaling),
                               'missing_value': ohc_per_m2.fill_value,
                               'notes': notes_text} 
    gio.set_dim_atts(dset_out, str(time_coord.units))

    outfile_metadata = {inargs.infile: cube.attributes['history']}

    gio.set_global_atts(dset_out, cube.attributes, outfile_metadata)
   
    dset_out.to_netcdf(inargs.outfile,)

In [ ]:
# Write output file
d = {}
for dim in darray.dims:
    d[dim] = darray[dim]
d[inargs.variable] = (darray.dims, zonal_anomaly)

dset_out = xray.Dataset(d)

dset_out[inargs.variable].attrs = {'long_name': darray.attrs['long_name'],
                                   'standard_name': darray.attrs['standard_name'],
                                   'units': darray.attrs['units'],
                                   'notes': 'The zonal mean has been subtracted at each time step.'}

gio.set_global_atts(dset_out, dset_in.attrs, {inargs.infile: dset_in.attrs['history'],})
dset_out.to_netcdf(inargs.outfile)

 dset_out['ohc'].encoding['_FillValue'] = ohc_per_m2.fill_value